# Kubetnetes 101

by Peter Nagy

## Part 1 - Explore the cluster (in team)

### List nodes

```bash
$ kubectl get nodes
```
```bash
ip-10-201-55-128.eu-central-1.compute.internal   Ready     master    5h        v1.11.10
ip-10-201-83-130.eu-central-1.compute.internal   Ready     node      5h        v1.11.10
```

### List namespaces

```bash
$ kubectl get ns
```
```bash
NAME          STATUS    AGE
dani          Active    44m
default       Active    5h
fpeti         Active    43m
interview     Active    2h
karcsi        Active    43m
kube-public   Active    5h
kube-system   Active    5h
pepo          Active    43m
tomi          Active    43m
```

### etc

## Part 2 - Deploy a few services and ingress controller

### Create secrets for database

In the next fews steps we will deploy a database based on this image: https://cloud.docker.com/repository/docker/nagypeter/k8s-database. Please examine the description on dockerhub.

As you can see there are a few requirements:  
**ENV:**  
 - **MYSQL_USER**="arbitrary username you will use by the service"
 - **MYSQL_PASSWORD**="arbitrary password for your user"
 - **MYSQL_ROOT_PASSWORD**="arbitrary root password"
 
**Args**:  
- --default-authentication-plugin=mysql_native_password

- Default port: 3306

Run the following command to create the secret containing the basis for our db deployment:

```bash
kubectl create secret generic mysql-secret --from-literal=mysql.user=tc2 --from-literal=mysql.password=secret --from-literal=mysql.root.password=supersecret -n YOUR_NAMESPACE
```

Lets create a db deployment (mysql-deployment.yaml):

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: database-deployment
  labels:
    app: database
  namespace: YOUR_NAMESPACE
spec:
  replicas: 1
  selector:
    matchLabels:
      app: database
  template:
    metadata:
      labels:
        app: database
    spec:
      containers:
      - name: database
        image: WHAT_COULD_BE_THE_IMAGE_NAME
        ports:
        - containerPort: WHAT_COULD_BE_THE_PORT
        env:
        - name: MYSQL_USER
          valueFrom:
            secretKeyRef:
              name: mysql-secret
              key: WHAT_COULD_BE_THE_KEY
        - name: WHAT_COULD_BE_THE_ENV_NAME
          valueFrom:
            secretKeyRef:
              name: mysql-secret
              key: mysql.password
        - name: MYSQL_ROOT_PASSWORD
          valueFrom:
            secretKeyRef:
              name: WHAT_COULD_BE_THE_SECRET_NAME
              key: mysql.root.password
        args:
        - --WHAT_COULD_BE_THE_ARG
```

Next, we will create a service from the deployment, so our node application will be able to reference the db.

```bash
apiVersion: v1
kind: Service
metadata:
  labels:
    app: database-service
  name: database-service
  namespace: YOUR_NAMESPACE
spec:
  ports:
  - protocol: TCP
    port: 3306
    targetPort: WHAT_COULD_BE_THE_TARGET_PORT
  selector:
    app: WHAT_COULD_BE_THE_SELECTOR
```

Now we will a create a node application deployment and service, which will be based on this image: https://cloud.docker.com/repository/docker/nagypeter/k8s-service

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: node-service-deployment
  labels:
    app: node-app
  namespace: YOUR_NAMESPACE
spec:
  replicas: 1
  selector:
    matchLabels:
      app: node-app
  template:
    metadata:
      labels:
        app: node-app
    spec:
      containers:
      - name: node-app
        image: https://cloud.docker.com/repository/docker/nagypeter/k8s-servic
        ports:
        - containerPort: 9000
        env:
        - name: MYSQL_HOST
          value: WHAT_COULD_BE_THE_VALUE
        - name: MYSQL_USERNAME
          valueFrom:
            secretKeyRef:
              name: mysql-secret
              key: mysql.user
        - name: MYSQL_PASSWORD
          valueFrom:
            secretKeyRef:
              name: mysql-secret
              key: mysql.password
        - name: MYSQL_DATABASE
          value: tc2
---
apiVersion: v1
kind: Service
metadata:
  labels:
    app: node-service
  name: node-service
  namespace: YOUR_NAMESPACE
spec:
  ports:
  - protocol: TCP
    port: 80
    targetPort: 9000
  selector:
    app: node-app
  type: NodePort    <--- ask my why this is NodePort
```

Even if you did everyhing right, the pod creation will fail. What could be the problem? Use the following command to figure it out:

```bash
kubectl describe get po

node-app-95f8bc4cb-ls4fd
```
```bash
kubectl describe po node-app-95f8bc4cb-ls4fd

...
```

Next, we will expose our service with an ingress controller, which will create an ALB for us.

```bash
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: "node-app-ingress"
  namespace: YOUR_NAMESPACE
  annotations:
    kubernetes.io/ingress.class: alb
    alb.ingress.kubernetes.io/scheme: internet-facing
    alb.ingress.kubernetes.io/listen-ports: '[{"HTTP": 80}]'
  labels:
    app: node-app-ingress
spec:
  rules:
    - host: k8s-your-namespace.tc2-edu.com
    - http:
        paths:
          - path: /*
            backend:
              serviceName: WHAT_COULD_BE_THE_SERVICE_NAME
              servicePort: 80
```

Lets see, if the alb-ingress-controller thinks our resource is OK:

```bash
kubectl get po -n kube-system | grep -i ingress
alb-ingress-controller-66f8c48776-49rpw                                  1/1       Running   0          5h
```
```bash
kubectl logs alb-ingress-controller-66f8c48776-49rpw  -n kube-system

I0716 17:02:46.179319       1 rules.go:77] interview/node-app-ingress: rule 1 created with conditions [{    Field: "path-pattern",    Values: ["/*"]  }]
I0716 17:02:46.265945       1 association.go:224] interview/node-app-ingress: creating securityGroup cd5608a6-interview-nodeapp-230b:managed LoadBalancer securityGroup by ALB Ingress Controller
I0716 17:02:46.341787       1 tags.go:69] interview/node-app-ingress: modifying tags {  kubernetes.io/cluster-name: "kubernetes-generic-pzn2zg7f.internal",  kubernetes.io/namespace: "interview",  kubernetes.io/ingress-name: "node-app-ingress"} on sg-06559161f2b0d228b
I0716 17:02:46.429567       1 security_group.go:50] interview/node-app-ingress: granting inbound permissions to securityGroup sg-06559161f2b0d228b: [{    FromPort: 80,    IpProtocol: "tcp",    IpRanges: [{        CidrIp: "0.0.0.0/0",        Description: "Allow ingress on port 80 from 0.0.0.0/0"      }],    ToPort: 80  }]
I0716 17:02:46.561258       1 lb_attachment.go:30] interview/node-app-ingress: modify securityGroup on LoadBalancer arn:aws:elasticloadbalancing:eu-central-1:471509798482:loadbalancer/app/cd5608a6-interview-nodeapp-230b/191556c4f21d31aa to be [sg-06559161f2b0d228b]
I0716 17:02:46.799674       1 association.go:224] interview/node-app-ingress: creating securityGroup instance-cd5608a6-interview-nodeapp-230b:managed instance securityGroup by ALB Ingress Controller
I0716 17:02:46.874682       1 tags.go:69] interview/node-app-ingress: modifying tags {  kubernetes.io/ingress-name: "node-app-ingress",  kubernetes.io/cluster-name: "kubernetes-generic-pzn2zg7f.internal",  kubernetes.io/namespace: "interview"} on sg-037bcdbe6d8758312
I0716 17:02:46.976611       1 security_group.go:50] interview/node-app-ingress: granting inbound permissions to securityGroup sg-037bcdbe6d8758312: [{    FromPort: 0,    IpProtocol: "tcp",    ToPort: 65535,    UserIdGroupPairs: [{        GroupId: "sg-06559161f2b0d228b"      }]  }]
```

Find the alb DNS to check our service, look for the ALB which's name contains your namespace:

```bash
aws elbv2 describe-load-balancers --region eu-central-1
```

Lets make a call:

```bash
curl -H 'Host: k8s-your-namespace.tc2.com' 'ALB-DNS-NAME'
```

So what message do you see?